# Project 2: Data Modeling with Cassandra

## Purpose:  
### Sparkify would like a data engineer to create an Apache Cassandra database which can create queries on song play data to answer the questions posed by the data analysis team.

## Project Overview:  
### Data will be modeled by creating tables and queries to perform the queries requested by the analysis team.  Data will be inserted into these tables from a unified CSV file that has combined the events from CSV files in the event_data directory.

-------------------------------------

# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [3]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [4]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [5]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
except Exception as e:
    print(e)

#### Set Keyspace

In [6]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

### 1. Query Goal: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [7]:
## Create Query 1 and the session_history table

query1 = """SELECT artist, song, length FROM session_history WHERE sessionId = 338 AND itemInSession = 4
"""

# PK design rationale:  The query is selecting by sessionId and itemInSession. Make the partition key sessionId so the index can first
# find all the rows for the sessionId, and then add itemInSession as a clustering key so that the index can find the correct item in the
# session.

query = "CREATE TABLE IF NOT EXISTS session_history "
query = query + "(sessionId INT, itemInSession INT, artist TEXT, song TEXT, length DECIMAL, PRIMARY KEY(sessionId, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [8]:
## Insert rows into session_history

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_history (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [9]:
## Verify the data was entered into the table

try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### 2. Query Goal: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [10]:
## Create Query 2 and the user_play_history table

query2 = """SELECT artist, song, firstName, lastName FROM user_play_history WHERE userId = 10 AND sessionId = 182
"""

# PK design rationale:  The query is selecting by userId and sessionId. Make the partition key the combination of userId
# and sessionId so that the session data for each user is co-located on a node for faster access. Add itemInSession as a
# clustering key so that the rows will be sorted by itemInSession.

query = "CREATE TABLE IF NOT EXISTS user_play_history "
query = query + "(userId INT, sessionId INT, itemInSession INT, artist TEXT, song TEXT, firstName TEXT, lastName TEXT, PRIMARY KEY((userId, sessionId), itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [11]:
## Insert rows into user_play_history

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_play_history (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [12]:
## Verify the data was entered into the table

try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### 3. Query Goal: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [14]:
## Create Query 3 and the song_play_history table

query3 = """SELECT firstName, lastName FROM song_play_history WHERE song = 'All Hands Against His Own'
"""

# PK design rationale:  The query is selecting by song. Make the partition key song so the index can first
# find all the rows for the song. Note that making a primary key based off of song alone is not enough, as
# multiple users can play this song.  To make the primary key specific enough to return all users that
# played that song, the primary key needs to include the userId.

query = "CREATE TABLE IF NOT EXISTS song_play_history "
query = query + "(song TEXT, userId INT, firstName TEXT, lastName TEXT, PRIMARY KEY(song, userId))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [17]:
## Insert rows into song_play_history

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_play_history (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [18]:
## Verify the data was entered into the table

try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [19]:
query = "DROP TABLE IF EXISTS session_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS user_play_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS song_play_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()